<a href="https://colab.research.google.com/github/kvinne-anc/Tanzania-Soil-Topography-Waterpumps-Project/blob/main/TanzaniaGEO_LogRegModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 245kB 4.4MB/s 
     |████████████████████████████████| 3.1MB 7.7MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 1.1MB 33.5MB/s 
     |████████████████████████████████| 112kB 46.8MB/s 
     |████████████████████████████████| 296kB 45.0MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27099 sha256=cbd20c6fe33e220db8da09c36de2020527e7240f34bd25d9d6fdf068a4dc42ae
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107437 sha256=3a5a568957a5ba6a9f3ee5f87cbcb1a91ea7362e9ff02308d6632f5b55b4ccb9
  Stored in directory: /root/.cache/pip/wheels/c0/a3/b0/f27b1cfe32ea131a3715169132ff6d85653789e80e966c3bf6
Successfully built htmlmin phik
ERROR: goo

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (2,450 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../python3-numpy_

In [ ]:

import pandas as pd
import geopandas as gdp 
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn.model_selection import train_test_split
import sys

In [ ]:
pumps = pd.read_csv('/content/new_tza_waterpumps.csv')
labels = pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv')


ParserError: ignored

,24058532714844,34.33818817138672 -4.819189071655273,34.34302139282227 -4.809428215026855,34.34297561645508 -4.779298305511475,34.33648300170898 -4.758941173553467,34.32677841186523 -4.757298469543457,34.28800201416016 -4.774341583251953,34.27829742431641 -4.772695541381836,34.25322341918945 -4.757996559143066,34.24434280395508 -4.761241912841797,34.24274444580078 -4.772641181945801,34.27273941040039 -4.843512535095215,34.26709365844727 -4.851646423339844,34.25820159912109 -4.850813388824463,34.24848937988281 -4.8459153175354,34.2202262878418 -4.867846488952637,34.21457290649414 -4.877600193023682,34.20973587036133 -4.885727882385254,34.20489883422852 -4.894674777984619,34.19924926757812 -4.901995658874512,34.19441223144531 -4.911752700805664,34.19118499755859 -4.921509742736816,34.18634796142578 -4.931266307830811,34.18069839477539 -4.940207004547119,34.17424011230469 -4.947522640228271,34.16535186767578 -4.949134349822998,34.16049194335938 -4.93853759765625,34.16209030151367 -4.927961826324463,34.16369247436523 -4.918195724487305,34.16368103027344 -4.905989170074463,34.16285705566406 -4.892965793609619,34.16608428955078 -4.881569862365723,34.19917297363281 -4.840131759643555,34.19997024536133 -4.83035945892334,34.19834136962891 -4.820590972900391,34.17487335205078 -4.79774808883667,34.17243957519531 -4.788784503936768,34.17323684692383 -4.779018878936768,34.21844863891602 -4.735957622528076,34.22086334228516 -4.726191997528076,...,34.19171524047852 -4.673216342926025,34.18927383422852 -4.664261341094971,34.18357086181641 -4.625976085662842,34.17629623413086 -4.618640422821045,34.13589096069336 -4.612046718597412,34.12942123413086 -4.60389518737793,34.19154739379883 -4.533167362213135,34.19234466552734 -4.522579193115234,34.19070816040039 -4.511990547180176,34.19151306152344 -4.502222537994385,34.19958114624023 -4.494901180267334,34.20846176147461 -4.49247407913208,34.21412658691406 -4.501440048217773,34.21817779541016 -4.510415077209473,34.22545623779297 -4.516939163208008,34.23433303833008 -4.512062549591064,34.23836517333984 -4.503923892974854,34.23916244506836 -4.494967460632324,34.2391471862793 -4.485195159912109,34.24479293823242 -4.475435256958008,34.25205612182617 -4.469746589660645,34.2577018737793 -4.461607933044434,34.25849151611328 -4.452650547027588,34.2512092590332 -4.443676471710205,34.24312973022461 -4.437966823577881,34.2447395324707 -4.428194046020508,34.25437927246094 -4.392378807067871,34.26244735717773 -4.387494087219238,34.27132797241211 -4.385884284973145,34.27778625488281 -4.376927852630615,34.27852630615234 -4.328063488006592,34.25508499145508 -4.310112953186035,34.28166198730469 -4.257199764251709,34.27111053466797 -4.218892097473145,34.25092315673828 -4.215616226196289,34.16861343383789 -4.249701499938965,34.16456604003906 -4.233400344848633,34.17343902587891 -4.229350566864014,34.24038314819336 -4.17730712890625,"34.2554817199707 -4.150426864624023))"""
0,50318.0,27377.0,0.0,2013-01-17,Rwssp,1170.0,DWE,34.642644,-4.054159,Nyegere,0.0,Internal,Nyegere,Singida,13.0,1.0,Iramba,Ibaga,245.0,True,GeoData Consultants Ltd,VWC,NaN,True,2009.0,afridev,afridev,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,...,0.186032,7.419713,LPe2,Eutric Leptosols,"POLYGON ((34.2554817199707 -4.150426864624023,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51288.0,52689.0,50.0,2013-01-16,Tasaf,1088.0,SHY BUILDERS,34.339135,-4.117980,Msele,0.0,Internal,Msele,Singida,13.0,1.0,Iramba,Tulya,223.0,True,GeoData Consultants Ltd,VWC,None,True,2008.0,india mark ii,india mark ii,handpump,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,...,0.186032,7.419713,LPe2,Eutric Leptosols,"POLYGON ((34.2554817199707 -4.150426864624023,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
train = pd.merge(pd.read_csv('/content/tza_waterpumps.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

test = pd.read_csv('/content/new_test_waterpumps1.csv')

sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

MergeError: ignored

In [ ]:

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

KeyError: ignored

In [ ]:
train.shape, val.shape, test.shape

NameError: ignored

In [ ]:
train['status_group'].value_counts(normalize=True)

functional                 0.545522
non functional             0.386158
functional needs repair    0.068321
Name: status_group, dtype: float64

In [ ]:
cat_col = train.columns[train.dtypes=='object']

(cat_col)

Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group', 'SOIL_TYPE',
       'MAIN_TYPE', 'geometry', 'status_group'],
      dtype='object')

In [ ]:
train[cat_col] = train[cat_col].astype('category')

In [ ]:
train.loc[0]
train['lga'].cat.codes

2672      88
49889    110
52171     96
45347    121
53494     57
        ... 
52516    123
3278      11
31958     16
9083     103
41732     12
Length: 45989, dtype: int8

In [ ]:
trained = train.drop(['amount_tsh', 'index_left', 'AREA', 'PERIMETER', 'installer', 'extraction_type', 'construction_year', 'quantity', 'wpt_name', 'num_private', 'basin', 'region', 'district_code', 'population', 'public_meeting', 'recorded_by', 'id', 'scheme_management', 'scheme_name', 'permit', 'extraction_type_group', 'extraction_type_class', 'management_group', 'management', 'payment', 'payment_type', 'quality_group', 'source_type', 'source_class', 'MAIN_TYPE', 'source', 'date_recorded', 'waterpoint_type', 'waterpoint_type_group', 'status_group'], axis=1)

In [ ]:
trained.head()

,funder,gps_height,longitude,latitude,subvillage,region_code,lga,ward,water_quality,quantity_group,SOIL_TYPE,geometry
2672,Oxfam,0.0,30.691264,-2.847368,Nyakuguma,18.0,Ngara,Bugarama,soft,enough,FRh16,POLYGON ((30.87990951538086 -0.997727453708648...
49889,Mission,1466.0,35.297519,-9.760209,Ganga,10.0,Songea Rural,Wino,soft,enough,AChX4,POLYGON ((36.40432739257812 -8.525897026062012...
52171,Watu Wa Ujerumani,1188.0,34.031656,-1.146791,Kabwana,20.0,Rorya,Mkoma,soft,enough,CMx3,POLYGON ((34.09883117675781 -1.023067951202393...
45347,Dhv,379.0,36.861544,-9.022433,Mwanza,5.0,Ulanga,Mbuga,unknown,enough,CMo21,POLYGON ((37.00050354003906 -7.480969429016113...
53494,Netherland,0.0,33.754734,-3.157654,Nyanhumbili,17.0,Maswa,Buchambi,soft,enough,PLe7,"POLYGON ((33.6700325012207 -3.178311109542847,..."


In [ ]:
numeric_features = trained.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = trained.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 10000
categorical_features = cardinality[cardinality <= 1000000].index.tolist()

# Combine the lists 

features = numeric_features + categorical_features
print(features)

['gps_height', 'longitude', 'latitude', 'region_code', 'funder', 'subvillage', 'lga', 'ward', 'water_quality', 'quantity_group', 'SOIL_TYPE', 'geometry']


In [ ]:
target = train['status_group']

In [ ]:
X_train = trained[features]
y_train = train['status_group']
X_val = val[features]
y_val = val['status_group']
X_test = test[features]

In [ ]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    StandardScaler(), 
    LogisticRegression(multi_class='auto', solver='lbfgs', n_jobs=-1)
)

# Fit on train
pipeline.fit(X_train, y_train)

# Score on val
print('Validation Accuracy', pipeline.score(X_val, y_val))

# Predict on test
y_pred = pipeline.predict(X_test)

TerminatedWorkerError: ignored